In [ ]:
!pip install ultralytics -q

# **YOLOv8 Orig**

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import shutil


In [ ]:
# Cấu hình đường dẫn
model = YOLO('yolov8n.pt')  # YOLOv8 nano pretrained trên COCO
test_dir = Path('/content/drive/MyDrive/Khóa Luận/Data_Object/testing/image_2')
out_dir  = Path('/content/drive/MyDrive/Khóa Luận/kitti_submit_yolov8_orig')
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
#  Mappings: COCO index → KITTI class name
coco2kitti = {
    2: 'Car',
    0: 'Pedestrian',
    1: 'Cyclist'
}

In [ ]:
# Predict & ghi file .txt theo KITTI format
for img_path in sorted(test_dir.glob('*.*')):
    # predict single image
    res = model.predict(source=str(img_path), save=False, verbose=False)[0]
    boxes   = res.boxes.xyxy.cpu().numpy()        # [N,4]
    classes = res.boxes.cls.cpu().numpy().astype(int)  # [N,]
    confs   = res.boxes.conf.cpu().numpy()        # [N,]

    txt_path = out_dir / f"{img_path.stem}.txt"
    with open(txt_path, 'w') as f:
        for (x1,y1,x2,y2), c, p in zip(boxes, classes, confs):
            # Bỏ qua nếu không phải Car/Pedestrian/Cyclist
            if c not in coco2kitti:
                continue
            cls_name = coco2kitti[c]
            # KITTI submission format (16 trường):
            line = (
                f"{cls_name} 0 0 0 "
                f"{x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f} "
                "0 0 0 0 0 0 0 "
                f"{p:.4f}"
            )
            f.write(line + "\n")

print(f"Đã ghi xong {len(list(out_dir.glob('*.txt')))} files vào {out_dir}")

In [ ]:
# Nén thành ZIP để submit
shutil.make_archive('kitti_predictions_yolov8_orig', 'zip', root_dir=str(out_dir))
print(f"File ZIP đã tạo: kitti_predictions_yolov8_orig.zip")